In [ ]:
import os.path as osp
import sys
import random

path = osp.dirname(osp.abspath(''))
sys.path.append(path)
sys.path.append(osp.join(path, "./open_biomed"))
path

In [ ]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:

def read_text(path=None):
    with open(path, 'r') as f:

        text_list = []
        for index, line in enumerate(f):
            text = line.strip()
            text_list.append(text)
    return text_list
path='./text_after_openai_chemistry_extracted_cleaned.txt'
extracted_text=read_text(path=path)
path='./text_after_structure_filter.txt'
texts=read_text(path=path)

# path='./smis_after_structure_filter.txt'
# smis=read_text(path=path)

In [ ]:
def structure_isomers(path=''):
    sim_list=[]
    with open(path, 'r') as f:
        for index, line in enumerate(f):

            isomers_list = line.strip().split("\t")
            sim_list.append(isomers_list[0])
    return sim_list
structure_isomers=structure_isomers(path='./structure_ismers.txt')

smis=structure_isomers
print((len(texts),len(smis)))

In [ ]:
len(extracted_text),len(texts),len(smis)

In [ ]:
# 线性核函数，用 PyTorch 实现
def linear_kernel(X):
    return torch.mm(X, X.T)  # 使用矩阵乘法

# 居中核矩阵
def center_kernel(K):
    n = K.shape[0]
    H = torch.eye(n, device=K.device) - torch.ones((n, n), device=K.device) / n  # 中心化矩阵 H
    return torch.mm(H, torch.mm(K, H))

# CKA 实现，计算两个张量 X 和 Y 的 CKA
def CKA(X, Y):
    # 计算线性核矩阵
    K = linear_kernel(X)
    L = linear_kernel(Y)

    # 居中核矩阵
    Kc = center_kernel(K)
    Lc = center_kernel(L)

    # 计算 HSIC 值
    hsic_XY = torch.trace(torch.mm(Kc, Lc))
    hsic_XX = torch.trace(torch.mm(Kc, Kc))
    hsic_YY = torch.trace(torch.mm(Lc, Lc))

    # 计算 CKA 值
    cka_value = hsic_XY / torch.sqrt(hsic_XX * hsic_YY)




    return round(cka_value.item(),3)

    

In [ ]:
from metrics import cka,gram_linear
# structure_feats_np = structure_feats[indes].cpu().detach().numpy()
# text_feats_np = text_feats[indes].cpu().detach().numpy()
# # Calculate Gram matrices for structure_feats and text_feats
def de_biased_cka(structure_feats,text_feats):
    structure_feats_np = structure_feats.cpu().detach().numpy()
    text_feats_np = text_feats.cpu().detach().numpy()
    gram_structure = gram_linear(structure_feats_np)
    gram_text = gram_linear(text_feats_np)

    # Calculate CKA between the two Gram matrices
    cka_value = cka(gram_structure, gram_text, debiased=True)

    return round(cka_value,3)

In [ ]:
import torch.nn.functional as F

def retrieve(A, B, K=1):
    """
    检索任务函数，输入两个 tensor A 和 B，返回 A 和 B 匹配的索引值、匹配的分数和正确匹配的条目数。

    参数：
    - A: torch.Tensor，待匹配的 query tensor，形状 (n, d)。
    - B: torch.Tensor，待匹配的 target tensor，形状 (n, d)。
    - K: int，检索 top K 匹配。

    返回：
    - indices: 每个 A 对应的最相似 B 的索引值，形状 (n, K)。
    - scores: 每个 A 对应的最相似 B 的分数，形状 (n, K)。
    - correct_total: 匹配正确的总条目数。
    """

    # 计算 A 和 B 之间的余弦相似度 (n, n)
    similarities = F.cosine_similarity(A.unsqueeze(1), B.unsqueeze(0), dim=-1)
    # 获取相似度最高的前 K 个条目的索引和分数
    topk_scores, topk_indices = torch.topk(similarities, K, dim=1)
    # 计算总的正确匹配数量
    # 正确的定义：A[i] 和 B[i] 相匹配，意味着第 i 个 A 应该匹配第 i 个 B
    correct_total = 0
    n = A.size(0)
    for i in range(n):
        # 检查 top K 的索引是否包含正确匹配
        if i in topk_indices[i]:
            correct_total += 1
    return topk_indices, topk_scores, correct_total

In [ ]:
def overleap(indexA,indexB ):
    C=[]
    D=[]
    for a, b in zip(indexA, indexB):
        overlap = list(set(a) & set(b))
        C.append(overlap)
        D.append([len(overlap)])

    # Print results
    print("Overlap C:", C)
    print("Number of overlaps D:", D)

    sum=0
    for i in C:
        sum=sum+len(i)
    print(sum)
    jaccard_similarity=0
    for a, b in zip(indexA, indexB):
        jaccard_similarity = jaccard_similarity+len(set(a) & set(b)) / len(set(a) | set(b))

    print("Mean Jaccard Similarity:", round(jaccard_similarity/len(indexA),3))

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

def KM(embedding=None,num_clusters=5):
    num_clusters = num_clusters  # You can set this based on your requirement

    # Initialize and fit the KMeans model
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(embedding)

    return kmeans

In [ ]:
from sklearn.neighbors import NearestNeighbors

def NN(embedding=None,num=10):
    # Initialize the Nearest Neighbors model
    num_neighbors = num  # Number of nearest neighbors to find
    nn_model = NearestNeighbors(n_neighbors=num_neighbors)

    # Fit the model to your data
    try:
        nn_model.fit(embedding.cpu().numpy())
    except:
        nn_model.fit(embedding.detach().cpu().numpy())

    return nn_model

In [ ]:
len(extracted_text),len(smis),len(texts)

## MoLFormer

In [ ]:
sim_embeddings=torch.load('./savedpt/Structure_sim_embeddings.pt')

In [ ]:
sim_embeddings.pooler_output.shape

In [ ]:
# !pip install git+https://github.com/lightonai/pylate.git
# !pip install git+https://github.com/UKPLab/sentence-transformers.git
# !pip install git+https://github.com/huggingface/transformers.git

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# Define model ID
model_id = "answerdotai/ModernBERT-base" 

# Load tokenizer and model, move model to GPU if available
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Tokenize input texts with padding for batch processing
inputs = tokenizer(texts,padding=True,truncation=True, return_tensors="pt")

# Move inputs to GPU if available
inputs = {key: value.to(device) for key, value in inputs.items()}

# Forward pass through the model
with torch.no_grad():  # Disable gradient computation for inference
    outputs = model(**inputs)

# Extract the last hidden states
last_hidden_states = outputs.last_hidden_state

# Print the shape of the last hidden states
print("Last hidden states shape:", last_hidden_states.shape)

# 取 [CLS] token 的 embedding（句子嵌入）
cls_embedding = last_hidden_states[:, 0, :]

# 也可以取所有 token 的平均值作为句子嵌入
mean_embedding = torch.mean(last_hidden_states, dim=1)

print(f"CLS token embedding size: {cls_embedding.shape}")
print(f"Mean token embedding size: {mean_embedding.shape}")

print(cls_embedding.shape,mean_embedding.shape)

In [ ]:
# Tokenize input texts with padding for batch processing
extracted_inputs = tokenizer(extracted_text, padding=True,truncation=True, return_tensors="pt")

# Move inputs to GPU if available
extracted_inputs = {key: value.to(device) for key, value in extracted_inputs.items()}

# Forward pass through the model
with torch.no_grad():  # Disable gradient computation for inference
    extracted_outputs = model(**extracted_inputs)

# Extract the last hidden states
extracted_last_hidden_states = extracted_outputs.last_hidden_state

# Print the shape of the last hidden states
print("Last hidden states shape:", extracted_last_hidden_states.shape)

# 取 [CLS] token 的 embedding（句子嵌入）
extracted_cls_embedding = extracted_last_hidden_states[:, 0, :]

# 也可以取所有 token 的平均值作为句子嵌入
extracted_mean_embedding = torch.mean(extracted_last_hidden_states, dim=1)

print(f"CLS token embedding size: {extracted_cls_embedding.shape}")
print(f"Mean token embedding size: {extracted_mean_embedding.shape}")

print(extracted_cls_embedding.shape,extracted_mean_embedding.shape)

In [ ]:
SCIB_cls_embedding=cls_embedding
SCIB_mean_embedding=mean_embedding
SCIB_extracted_text_mean_embedding=extracted_mean_embedding
SCIB_extracted_text_cls_embedding=extracted_cls_embedding

#### CKA

In [ ]:
de_biased_cka(SCIB_cls_embedding,sim_embeddings.pooler_output),de_biased_cka(SCIB_mean_embedding,sim_embeddings.pooler_output),de_biased_cka(SCIB_extracted_text_cls_embedding,sim_embeddings.pooler_output),de_biased_cka(SCIB_extracted_text_mean_embedding,sim_embeddings.pooler_output)

In [ ]:
CKA(SCIB_cls_embedding.cpu(),sim_embeddings.pooler_output),CKA(SCIB_mean_embedding.cpu(),sim_embeddings.pooler_output),CKA(SCIB_extracted_text_cls_embedding.cpu(),sim_embeddings.pooler_output),CKA(SCIB_extracted_text_mean_embedding.cpu(),sim_embeddings.pooler_output)

### Now, we have cls mean embedding of texts and extracted_text
cls_embedding,

mean_embedding,

extracted_text_cls_embedding,

extracted_text_mean_embedding,

#### text NN

In [ ]:
SCIB_texts_cls_NN=NN(embedding=SCIB_cls_embedding)
SCIB_texts_mean_NN=NN(embedding=SCIB_mean_embedding)
SCIB_extracted_text_cls_NN=NN(embedding=SCIB_extracted_text_cls_embedding)
SCIB_extracted_text_mean_NN=NN(embedding=SCIB_extracted_text_mean_embedding)

In [ ]:
SCIB_texts_cls_NN_distance,SCIB_texts_cls_NN_indes=SCIB_texts_cls_NN.kneighbors(sim_embeddings.pooler_output)
SCIB_texts_mean_NN_distance,SCIB_texts_mean_NN_indes=SCIB_texts_mean_NN.kneighbors(sim_embeddings.pooler_output)
SCIB_extracted_text_cls_NN_distance,SCIB_extracted_text_cls_NN_indes=SCIB_extracted_text_cls_NN.kneighbors(sim_embeddings.pooler_output)
SCIB_extracted_text_mean_NN_distance,SCIB_extracted_text_mean_NN_indes=SCIB_extracted_text_mean_NN.kneighbors(sim_embeddings.pooler_output)

In [ ]:
SCIB_texts_match_indices, SCIB_texts_match_scores, SCIB_texts_match_correct_total = retrieve(sim_embeddings.pooler_output, SCIB_cls_embedding.cpu().detach(), K=10)
SCIB_texts_mean_match_indices, SCIB_texts_mean_match_scores, SCIB_texts_mean_match_correct_total = retrieve(sim_embeddings.pooler_output, SCIB_mean_embedding.cpu().detach(), K=10)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", SCIB_texts_match_correct_total)
print("正确匹配的条目数：", SCIB_texts_mean_match_correct_total)
SCIB_extracted_text_match_indices, SCIB_extracted_text_match_scores, SCIB_extracted_text_match_correct_total = retrieve(sim_embeddings.pooler_output, SCIB_extracted_text_cls_embedding.cpu().detach(), K=10)
SCIB_extracted_text_mean_match_indices, SCIB_extracted_text_mean_match_scores, SCIB_extracted_text_mean_match_correct_total = retrieve(sim_embeddings.pooler_output, SCIB_extracted_text_mean_embedding.cpu().detach(), K=10)

print("正确匹配的条目数：", SCIB_extracted_text_match_correct_total)
print("正确匹配的条目数：", SCIB_extracted_text_mean_match_correct_total)

In [ ]:
SCIB_texts_k1_match_indices, SCIB_texts_k1_match_scores, SCIB_texts_k1_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), SCIB_cls_embedding.cpu().detach(), K=1)
SCIB_texts_K1_mean_match_indices, SCIB_texts_k1_mean_match_scores, SCIB_texts_k1_mean_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), SCIB_mean_embedding.cpu().detach(), K=1)

# 打印结果
# print("Top K 索引：", indices)
# print("Top K 分数：", scores)
print("正确匹配的条目数：", SCIB_texts_k1_match_correct_total)
print("正确匹配的条目数：", SCIB_texts_k1_mean_match_correct_total)
SCIB_extracted_k1_text_match_indices, SCIB_extracted_k1_text_match_scores, SCIB_extracted_k1_text_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), SCIB_extracted_text_cls_embedding.cpu().detach(), K=1)
SCIB_extracted_k1_text_mean_match_indices, SCIB_extracted_k1_text_mean_match_scores, SCIB_extracted_text_k1_mean_match_correct_total = retrieve(sim_embeddings.pooler_output.cpu().detach(), SCIB_extracted_text_mean_embedding.cpu().detach(), K=1)

print("正确匹配的条目数：", SCIB_extracted_k1_text_match_correct_total)
print("正确匹配的条目数：", SCIB_extracted_text_k1_mean_match_correct_total)

In [ ]:
overleap(SCIB_texts_match_indices.cpu().numpy(), SCIB_texts_cls_NN_indes)

overleap(SCIB_texts_mean_match_indices.cpu().numpy(), SCIB_texts_mean_NN_indes)
#_____________cross should get low score_____________

overleap(SCIB_texts_match_indices.cpu().numpy(), SCIB_texts_mean_NN_indes)
overleap(SCIB_texts_mean_match_indices.cpu().numpy(), SCIB_texts_cls_NN_indes)
print('='*20)
overleap(SCIB_extracted_text_match_indices.cpu().numpy(), SCIB_extracted_text_cls_NN_indes)
overleap(SCIB_extracted_text_mean_match_indices.cpu().numpy(), SCIB_extracted_text_mean_NN_indes)
#_____________cross should get low score_____________

overleap(SCIB_extracted_text_mean_match_indices.cpu().numpy(), SCIB_extracted_text_cls_NN_indes)
overleap(SCIB_extracted_text_match_indices.cpu().numpy(), SCIB_extracted_text_mean_NN_indes)
# 8562
# Mean Jaccard Similarity: 0.146

#### SMILES NN

In [ ]:
SCIBSMIS_texts_cls_NN=NN(embedding=sim_embeddings.pooler_output)
SCIBSMIS_texts_mean_NN=NN(embedding=sim_embeddings.pooler_output)
SCIBSMIS_extracted_text_cls_NN=NN(embedding=sim_embeddings.pooler_output)
SCIBSMIS_extracted_text_mean_NN=NN(embedding=sim_embeddings.pooler_output)

In [ ]:
SCIBSMIS_texts_cls_NN_distance,SCIBSMIS_texts_cls_NN_indes=SCIBSMIS_texts_cls_NN.kneighbors(SCIB_cls_embedding.cpu().numpy())
SCIBSMIS_texts_mean_NN_distance,SCIBSMIS_texts_mean_NN_indes=SCIBSMIS_texts_mean_NN.kneighbors(SCIB_mean_embedding.cpu().numpy())

SCIBSMIS_extracted_text_cls_NN_distance,SCIBSMIS_extracted_text_cls_NN_indes=SCIBSMIS_extracted_text_cls_NN.kneighbors(SCIB_extracted_text_cls_embedding.cpu().numpy())
SCIBSMIS_extracted_text_mean_NN_distance,SCIBSMIS_extracted_text_mean_NN_indes=SCIBSMIS_extracted_text_mean_NN.kneighbors(SCIB_extracted_text_mean_embedding.cpu().numpy())

In [ ]:
overleap(SCIB_texts_match_indices.cpu().numpy(), SCIBSMIS_texts_cls_NN_indes)

overleap(SCIB_texts_mean_match_indices.cpu().numpy(), SCIBSMIS_texts_mean_NN_indes)
#_____________cross should get low score_____________

overleap(SCIB_texts_match_indices.cpu().numpy(), SCIBSMIS_texts_mean_NN_indes)
overleap(SCIB_texts_mean_match_indices.cpu().numpy(), SCIBSMIS_texts_cls_NN_indes)
print('='*20)
overleap(SCIB_extracted_text_match_indices.cpu().numpy(), SCIBSMIS_extracted_text_cls_NN_indes)
overleap(SCIB_extracted_text_mean_match_indices.cpu().numpy(), SCIBSMIS_extracted_text_mean_NN_indes)

#_____________cross should get low score_____________
overleap(SCIB_extracted_text_mean_match_indices.cpu().numpy(), SCIBSMIS_extracted_text_cls_NN_indes)
overleap(SCIB_extracted_text_match_indices.cpu().numpy(), SCIBSMIS_extracted_text_mean_NN_indes)


### text K-MEANS

In [ ]:
SCIB_cls_KM=KM(SCIB_cls_embedding.cpu().numpy())
SCIB_mean_KM=KM(SCIB_mean_embedding.cpu().numpy())
SCIB_extracted_text_cls_KM=KM(SCIB_extracted_text_cls_embedding.cpu().numpy())
SCIB_extracted_text_mean_KM=KM(SCIB_extracted_text_mean_embedding.cpu().numpy())

In [ ]:
SCIB_cls_KM_index=SCIB_cls_KM.predict(SCIB_cls_embedding.cpu().numpy())
SCIB_mean_KM_index=SCIB_mean_KM.predict(SCIB_mean_embedding.cpu().numpy())
SCIB_extracted_text_cls_KM_index=SCIB_extracted_text_cls_KM.predict(SCIB_extracted_text_cls_embedding.cpu().numpy())
SCIB_extracted_text_mean_KM_index=SCIB_extracted_text_mean_KM.predict(SCIB_extracted_text_mean_embedding.cpu().numpy())

In [ ]:
SCIB_cls_match_index=SCIB_cls_KM.predict(SCIB_cls_embedding[SCIB_texts_k1_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())
SCIB_mean_match_index=SCIB_mean_KM.predict(SCIB_mean_embedding[SCIB_texts_K1_mean_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())
SCIB_et_cls_match_index=SCIB_extracted_text_cls_KM.predict(SCIB_extracted_text_cls_embedding[SCIB_extracted_k1_text_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())
SCIB_et_mean_match_index=SCIB_extracted_text_mean_KM.predict(SCIB_extracted_text_mean_embedding[SCIB_extracted_k1_text_mean_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())

In [ ]:
SCIB_x_cls=SCIB_cls_match_index==SCIB_cls_KM_index
SCIB_x_mean=SCIB_mean_KM_index==SCIB_mean_match_index
SCIB_x_et_cls=SCIB_extracted_text_cls_KM_index==SCIB_et_cls_match_index
SCIB_x_et_mean=SCIB_et_mean_match_index==SCIB_extracted_text_mean_KM_index

In [ ]:
np.sum(SCIB_x_cls),np.sum(SCIB_x_mean),np.sum(SCIB_x_et_cls),np.sum(SCIB_x_et_mean)

##### Smis K-MEANS

In [ ]:
SCIBSMIS_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
SCIBSMIS_mean_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
SCIBSMIS_extracted_text_cls_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
SCIBSMIS_extracted_text_mean_KM=KM(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
SCIBSMIS_cls_KM_index=SCIBSMIS_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
SCIBSMIS_mean_KM_index=SCIBSMIS_mean_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
SCIBSMIS_extracted_text_cls_KM_index=SCIBSMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))
SCIBSMIS_extracted_text_mean_KM_index=SCIBSMIS_extracted_text_mean_KM.predict(sim_embeddings.pooler_output.cpu().numpy().astype(np.float32))

In [ ]:
SCIBSMIS_cls_match_index=SCIBSMIS_cls_KM.predict(sim_embeddings.pooler_output[SCIB_texts_k1_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())
SCIBSMIS_mean_match_index=SCIBSMIS_mean_KM.predict(sim_embeddings.pooler_output[SCIB_texts_K1_mean_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())
SCIBSMIS_et_cls_match_index=SCIBSMIS_extracted_text_cls_KM.predict(sim_embeddings.pooler_output[SCIB_extracted_k1_text_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())
SCIBSMIS_et_mean_match_index=SCIBSMIS_extracted_text_mean_KM.predict(sim_embeddings.pooler_output[SCIB_extracted_k1_text_mean_match_indices].reshape(SCIB_cls_embedding.shape).cpu().numpy())

In [ ]:
SCIBSMIS_x_cls=SCIBSMIS_cls_match_index==SCIBSMIS_cls_KM_index
SCIBSMIS_x_mean=SCIBSMIS_mean_KM_index==SCIBSMIS_mean_match_index
SCIBSMIS_x_et_cls=SCIBSMIS_extracted_text_cls_KM_index==SCIBSMIS_et_cls_match_index
SCIBSMIS_x_et_mean=SCIBSMIS_et_mean_match_index==SCIBSMIS_extracted_text_mean_KM_index

In [ ]:
np.sum(SCIBSMIS_x_cls),np.sum(SCIBSMIS_x_mean),np.sum(SCIBSMIS_x_et_cls),np.sum(SCIBSMIS_x_et_mean)

## MolCLR

In [ ]:
gcn_embeddings=torch.load( './savedpt/ismers_gcn_embeddings.pth')
gcn_out_embeddings=torch.load( './savedpt/ismers_gcn_out_embeddings.pth')
gin_embeddings=torch.load( './savedpt/ismers_gin_embeddings.pth')
gin_out_embeddings=torch.load( './savedpt/ismers_gin_out_embeddings.pth')

In [ ]:
print(de_biased_cka(SCIB_cls_embedding, gcn_embeddings),de_biased_cka(SCIB_cls_embedding, gcn_out_embeddings),de_biased_cka(SCIB_cls_embedding,gin_embeddings),de_biased_cka(SCIB_cls_embedding,gin_out_embeddings))
print(de_biased_cka(SCIB_mean_embedding,gcn_embeddings),de_biased_cka(SCIB_mean_embedding,gcn_out_embeddings),de_biased_cka(SCIB_mean_embedding,gin_embeddings),de_biased_cka(SCIB_mean_embedding,gin_out_embeddings))
print(de_biased_cka(SCIB_extracted_text_cls_embedding, gcn_embeddings),de_biased_cka(SCIB_extracted_text_cls_embedding,gcn_out_embeddings),de_biased_cka(SCIB_extracted_text_cls_embedding,gin_embeddings),de_biased_cka(SCIB_extracted_text_cls_embedding,gin_out_embeddings))
print(de_biased_cka(SCIB_extracted_text_mean_embedding, gcn_embeddings),de_biased_cka(SCIB_extracted_text_mean_embedding,gcn_out_embeddings),de_biased_cka(SCIB_extracted_text_mean_embedding,gin_embeddings),de_biased_cka(SCIB_extracted_text_mean_embedding,gin_out_embeddings))

In [ ]:
print(CKA(SCIB_cls_embedding, gcn_embeddings.to(device)),CKA(SCIB_cls_embedding, gcn_out_embeddings.to(device)),CKA(SCIB_cls_embedding,gin_embeddings.to(device)),CKA(SCIB_cls_embedding,gin_out_embeddings.to(device)))
print(CKA(SCIB_mean_embedding,gcn_embeddings.to(device)),CKA(SCIB_mean_embedding,gcn_out_embeddings.to(device)),CKA(SCIB_mean_embedding,gin_embeddings.to(device)),CKA(SCIB_mean_embedding,gin_out_embeddings.to(device)))
print(CKA(SCIB_extracted_text_cls_embedding, gcn_embeddings.to(device)),CKA(SCIB_extracted_text_cls_embedding,gcn_out_embeddings.to(device)),CKA(SCIB_extracted_text_cls_embedding,gin_embeddings.to(device)),CKA(SCIB_extracted_text_cls_embedding,gin_out_embeddings.to(device)))
print(CKA(SCIB_extracted_text_mean_embedding, gcn_embeddings.to(device)),CKA(SCIB_extracted_text_mean_embedding,gcn_out_embeddings.to(device)),CKA(SCIB_extracted_text_mean_embedding,gin_embeddings.to(device)),CKA(SCIB_extracted_text_mean_embedding,gin_out_embeddings.to(device)))
